In [ ]:
def float_to_fixed_sign_magnitude(value, int_bits=0, frac_bits=15):
    total_bits = 1 + int_bits + frac_bits
    scale = 1 << frac_bits  # 2^frac_bits

    max_val = (1 << int_bits) - 1 + (1 - 1 / scale)
    min_val = -max_val

    if not (min_val <= value <= max_val):
        raise ValueError(f"Value {value} out of range for format with {int_bits} integer bits and {frac_bits} fractional bits.")

    # Get sign bit
    sign_bit = 0
    if value < 0:
        sign_bit = 1
        value = -value

    # Convert fractional part
    fixed_val = int(round(value * scale))

    # Mask to ensure it fits in the bits
    fixed_val &= (1 << (int_bits + frac_bits)) - 1

    # Shift left and add sign bit
    result = (sign_bit << (int_bits + frac_bits)) | fixed_val

    # Return binary string
    return format(result, f'0{total_bits}b')


In [ ]:
# Matrix A
print(float_to_fixed_sign_magnitude(0.25, 0, 15))
print(float_to_fixed_sign_magnitude(-0.65, 0, 15))
print(float_to_fixed_sign_magnitude(-0.1625, 0, 15))
print(float_to_fixed_sign_magnitude(0.3, 0, 15))
print(float_to_fixed_sign_magnitude(-0.4, 0, 15))
print(float_to_fixed_sign_magnitude(0.5, 1, 14))
print(float_to_fixed_sign_magnitude(-0.6, 1, 14))
print(float_to_fixed_sign_magnitude(0.7, 1, 14))
print(float_to_fixed_sign_magnitude(-0.8, 1, 14))
print(float_to_fixed_sign_magnitude(0.9, 1, 14))

0010000000000000
1101001100110011
1001010011001101
0010011001100110
1011001100110011
0010000000000000
1010011001100110
0010110011001101
1011001100110011
0011100110011010


In [ ]:
# Matrix B
print(float_to_fixed_sign_magnitude(-0.9, 1, 14))
print(float_to_fixed_sign_magnitude(0.8, 1, 14))
print(float_to_fixed_sign_magnitude(-0.7, 1, 14))
print(float_to_fixed_sign_magnitude(0.6, 1, 14))
print(float_to_fixed_sign_magnitude(-0.5, 1, 14))
print(float_to_fixed_sign_magnitude(0.4, 1, 14))
print(float_to_fixed_sign_magnitude(-0.3, 1, 14))
print(float_to_fixed_sign_magnitude(0.2, 1, 14))
print(float_to_fixed_sign_magnitude(-0.1, 1, 14))

1011100110011010
0011001100110011
1010110011001101
0010011001100110
1010000000000000
0001100110011010
1001001100110011
0000110011001101
1000011001100110


In [ ]:
def dec2q(x, a=0, b=15, out_format='bin'):
    """
    Convert decimal numbers to Qa.b fixed-point format in binary or hex (two's complement).

    Parameters:
    - x: Scalar or list of decimal numbers.
    - a: Integer bits (excluding sign bit), default 0.
    - b: Fractional bits, default 15.
    - out_format: 'bin' or 'hex', default 'bin'.

    Returns:
    - List of strings (binary or hex) in Qa.b format.
    """
    # Convert scalar to list
    if not isinstance(x, (list, tuple)):
        x = [x]

    # Validate inputs
    if a < 0 or b < 0:
        raise ValueError("Inputs 'a' and 'b' must be non-negative")
    out_format = out_format.lower()
    if out_format not in ['bin', 'hex']:
        raise ValueError("Format must be 'bin' or 'hex'")

    # Total bits: sign + integer + fractional
    N = 1 + a + b

    # For hex, check if bits are multiple of 4
    if out_format == 'hex' and N % 4 != 0:
        raise ValueError("Hex output requires total bits to be a multiple of 4")

    # Qa.b range
    max_val = (2 ** a) - (2 ** -b)
    min_val = -(2 ** a)

    y = []
    for value in x:
        # Clip input
        if value < min_val:
            value = min_val
            print("dec2q: input clipped (too small)")
        elif value > max_val:
            value = max_val
            print("dec2q: input clipped (too big)")

        # Scale to integer (two's complement)
        scale = 2 ** b
        if value >= 0:
            fixed_val = int(round(value * scale))
        else:
            fixed_val = int(round(abs(value) * scale))
            fixed_val = (1 << N) - fixed_val  # Two's complement

        # Ensure N-bit binary
        binary = format(fixed_val & ((1 << N) - 1), f'0{N}b')

        # Output format
        if out_format == 'hex':
            hex_val = format(int(binary, 2), f'0{N//4}x')
            y.append(hex_val)
        else:
            y.append(binary)

    return y

In [ ]:
print("Matrix A")
print(dec2q(0.1))
print(dec2q(-0.2))
print(dec2q(0.3))
print(dec2q(-0.4))
print(dec2q(0.5))
print(dec2q(-0.6))
print(dec2q(0.7))
print(dec2q(-0.8))
print(dec2q(0.9))

Matrix A
['0000110011001101']
['1110011001100110']
['0010011001100110']
['1100110011001101']
['0100000000000000']
['1011001100110011']
['0101100110011010']
['1001100110011010']
['0111001100110011']


In [ ]:
print("Matrix B")
print(dec2q(-0.9))
print(dec2q(0.8))
print(dec2q(-0.7))
print(dec2q(0.6))
print(dec2q(-0.5))
print(dec2q(0.4))
print(dec2q(-0.3))
print(dec2q(0.2))
print(dec2q(-0.1))

Matrix B
['1000110011001101']
['0110011001100110']
['1010011001100110']
['0100110011001101']
['1100000000000000']
['0011001100110011']
['1101100110011010']
['0001100110011010']
['1111001100110011']


In [ ]:
print(dec2q(-0.1))

['1111001100110011']


In [ ]:
print(dec2q(0.1))

['0000110011001101']
